# Using Python constructs to structure Signals

CoHDL supports Python classes, lists and dictionaries in synthesizable contexts. Similar to the previously discussed functions, these constructs are not translated into corresponding VHDL constructs like records or arrays. Their only purpose is to structure Signals/Variables and metadata.

Only operations that affect synthesizable objects show up in the generated HDL representation. It does not matter whether these objects are accessed via local variables, class members or list/dict items.

## classes

The core features of Python classes like member access, methods, operator overloading and inheritance are all allowed in synthesizable contexts. Methods are treated like all other functions and inlined at their call location.

In [1]:
from __future__ import annotations

from cohdl import Entity, Port, Signed
from cohdl import std

class Coord:
    def __init__(self, x, y):

        # adding members to classes is only possible
        # in the __init__ function
        self.x = x
        self.y = y
    
    def __add__(self, other: Coord):
        return Coord(self.x + other.x, self.y + other.y)

class ExampleCoord(Entity):
    a_x = Port.input(Signed[16])
    a_y = Port.input(Signed[16])
    b_x = Port.input(Signed[16])
    b_y = Port.input(Signed[16])

    sum_x = Port.output(Signed[16])
    sum_y = Port.output(Signed[16])

    def architecture(self):
        # python classes can be used to structure signals
        a = Coord(self.a_x, self.a_y)

        @std.concurrent
        def logic():
            # classes can also be instantiated in synthesizable contexts
            b = Coord(self.b_x, self.b_y)

            # operator overloading and member access are supported
            sum = a + b
            self.sum_x <<= sum.x
            self.sum_y <<= sum.y

print(std.VhdlCompiler.to_string(ExampleCoord))

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity ExampleCoord is
  port (
    a_x : in signed(15 downto 0);
    a_y : in signed(15 downto 0);
    b_x : in signed(15 downto 0);
    b_y : in signed(15 downto 0);
    sum_x : out signed(15 downto 0);
    sum_y : out signed(15 downto 0)
    );
end ExampleCoord;


architecture arch_ExampleCoord of ExampleCoord is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal buffer_sum_x : signed(15 downto 0);
  signal buffer_sum_y : signed(15 downto 0);
  signal x : signed(15 downto 0);
  signal y : signed(15 downto 0);
begin
  
  -- CONCURRENT BLOCK (buffer assignment)
  sum_x <= buffer_sum_x;
  sum_y <= buffer_sum_y;
  
  -- CONCURRENT BLOCK (logic)
  x <= (a_x) + (b_x);
  y <= (a_y) + (b_y);
  buffer_sum_x <= x;
  buffer_sum_y <= y;
end architecture arch_ExampleCoord;


## lists and dictionaries

Similar to classes, lists and dictionaries can be used and created in synthesizable contexts. It is not possible to add or remove elements after the initialization.

In [2]:
from cohdl import Entity, Port, BitVector, Unsigned, Bit
from cohdl import std

class ExampleComprehension(Entity):
    a = Port.input(Bit)
    b = Port.input(Bit)

    inp_bit = Port.input(Bit)
    inp_vec = Port.input(BitVector[4])
    inp_unsigned = Port.input(Unsigned[8])

    out_bit = Port.output(Bit)
    out_vec = Port.output(BitVector[4])
    out_unsigned = Port.output(Unsigned[8])

    result = Port.output(BitVector[3])

    def architecture(self):
        @std.concurrent
        def logic():
            # list/dict elements can have different types
            inp_list = [
                self.inp_bit,
                self.inp_vec,
                self.inp_unsigned,
            ]

            self.out_bit <<= inp_list[0]
            self.out_vec <<= inp_list[1]
            self.out_unsigned <<= inp_list[2]

            # lists/dicts can contain the results of expressions
            op_list = {
                "and": self.a & self.b,
                "or":  self.a | self.b,
                "xor": self.a ^ self.b
            }

            # the '@'-operator is used to concatenate Bits/BitVectors
            self.result <<= op_list["and"] @ op_list["or"] @ op_list["xor"]

print(std.VhdlCompiler.to_string(ExampleComprehension))

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity ExampleComprehension is
  port (
    a : in std_logic;
    b : in std_logic;
    inp_bit : in std_logic;
    inp_vec : in std_logic_vector(3 downto 0);
    inp_unsigned : in unsigned(7 downto 0);
    out_bit : out std_logic;
    out_vec : out std_logic_vector(3 downto 0);
    out_unsigned : out unsigned(7 downto 0);
    result : out std_logic_vector(2 downto 0)
    );
end ExampleComprehension;


architecture arch_ExampleComprehension of ExampleComprehension is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal buffer_out_bit : std_logic;
  signal buffer_out_vec : std_logic_vector(3 downto 0);
  signal buffer_out_unsigned : unsigned(7 downto 0);
  signal buffer_result : std_logic_vector(2 downto 0);
  signal temp : std_logic;
  signal temp_1 : std_logic;
  signal

## comprehensions

CoHDL allows list and dict comprehensions in synthesizable contexts as long as the number and types of the elements in the generated container are compile time constant.

In [3]:
from cohdl import Entity, Port, BitVector, Unsigned, select_with, Bit
from cohdl import std

class ExampleComprehension(Entity):
    index = Port.input(Unsigned[2])

    a = Port.input(BitVector[4])
    b = Port.input(BitVector[4])
    c = Port.input(BitVector[4])
    d = Port.input(BitVector[4])

    result_1 = Port.output(Bit)
    result_2 = Port.output(Bit)
    result_3 = Port.output(Bit)

    def architecture(self):
        @std.concurrent
        def logic():
            all_inp = [self.a, self.b, self.c, self.d]

            # the python builtins any and all are special cased in
            # CoHDL and translated into a chain of or/and expressions
            self.result_1 <<= any([inp[1] for inp in all_inp])
            self.result_2 <<= all([inp[3] for inp in all_inp])

            # use index to select
            # bit 0 from a,
            # bit 1 from b,
            # bit 2 from c or
            # bit 3 from d
            self.result_3 <<= select_with(
                self.index,
                {
                    nr: val[nr] for nr, val in enumerate(all_inp)
                }
            )

print(std.VhdlCompiler.to_string(ExampleComprehension))

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity ExampleComprehension is
  port (
    index : in unsigned(1 downto 0);
    a : in std_logic_vector(3 downto 0);
    b : in std_logic_vector(3 downto 0);
    c : in std_logic_vector(3 downto 0);
    d : in std_logic_vector(3 downto 0);
    result_1 : out std_logic;
    result_2 : out std_logic;
    result_3 : out std_logic
    );
end ExampleComprehension;


architecture arch_ExampleComprehension of ExampleComprehension is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal buffer_result_1 : std_logic;
  signal buffer_result_2 : std_logic;
  signal buffer_result_3 : std_logic;
  signal temp : boolean;
  signal temp_1 : boolean;
  signal temp_2 : std_logic;
begin
  
  -- CONCURRENT BLOCK (buffer assignment)
  result_1 <= buffer_result_1;
  result_2 <= buffer_result_2

## lazy evaluated return values

`selected_with`, if-expressions and functions with multiple return paths are a challenge for CoHDL because in Python these would return a runtime variable reference. When translated to VHDL these have to be mapped to a single object. For primitive types like Bits and BitVectors this can be solved by introducing temporary objects that are assigned a value based on the taken branch. This however does not work for user defined classes/lists or dictionaries because there is no VHDL equivalent for them.

To allow non-primitive return types, CoHDL uses a lazy evaluation approach. Whenever an expression produces a result where the corresponding Python object is ambiguous, CoHDL introduces a proxy object that keeps a reference to all possible sources. When members or elements are accesses all of these sources are checked to ensure, that all options exist and are compatible.

In [4]:
from __future__ import annotations

from cohdl import Entity, Port, Signed, Bit, select_with
from cohdl import std

class Coord:
    def __init__(self, x, y):

        # adding members to classes is only possible
        # in the __init__ function
        self.x = x
        self.y = y
    
    def __add__(self, other: Coord):
        return Coord(self.x + other.x, self.y + other.y)

class ExampleBranches(Entity):
    a_x = Port.input(Signed[16])
    a_y = Port.input(Signed[16])
    b_x = Port.input(Signed[16])
    b_y = Port.input(Signed[16])

    choose_a = Port.input(Bit)

    result_1 = Port.output(Signed[16])
    result_2 = Port.output(Signed[16])
    result_3 = Port.output(Signed[16])

    def architecture(self):
        a = Coord(self.a_x, self.a_y)

        @std.concurrent
        def logic_select():
            b = Coord(self.b_x, self.b_y)

            # the options of select_with can have arbitrary arguments
            # (even different types are allowed)
            # on its own this expression does nothing
            selected = select_with(
                self.choose_a,
                {
                    '1': a,
                    '0': b
                }
            )

            # only at this point is checked, that a member x
            # exists in all possible source objects
            # CoHDL creates a new temporary for this assignment
            # and inserts the required initialization at the location
            # of the select_with expression
            self.result_1 <<= selected.x
        
        @std.concurrent
        def logic_ifexpr():
            # if expressions are treated similar to select_with

            b = Coord(self.b_x, self.b_y)
            selected = a if self.choose_a else b
            self.result_2 <<= selected.x
        
        def choose_input():
            if self.choose_a:
                return a
            return Coord(self.b_x, self.b_y)
        
        @std.sequential
        def proc_fn():
            # when functions with multiple return paths are used
            # CoHDL inserts assignments to required temporaries at the location
            # of return statements
            selected = choose_input()
            self.result_3 <<= selected.x
            self.result_3 <<= selected.y

print(std.VhdlCompiler.to_string(ExampleBranches))

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity ExampleBranches is
  port (
    a_x : in signed(15 downto 0);
    a_y : in signed(15 downto 0);
    b_x : in signed(15 downto 0);
    b_y : in signed(15 downto 0);
    choose_a : in std_logic;
    result_1 : out signed(15 downto 0);
    result_2 : out signed(15 downto 0);
    result_3 : out signed(15 downto 0)
    );
end ExampleBranches;


architecture arch_ExampleBranches of ExampleBranches is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal buffer_result_1 : signed(15 downto 0);
  signal buffer_result_2 : signed(15 downto 0);
  signal buffer_result_3 : signed(15 downto 0);
  signal temp : signed(15 downto 0);
  signal temp_1 : boolean;
  signal temp_2 : signed(15 downto 0);
begin
  
  -- CONCURRENT BLOCK (buffer assignment)
  result_1 <= buffer_result_1;
  r